# How US Presidents Died According to Wikidata

<blockquote class="twitter-tweet" lang="en"><p lang="en" dir="ltr">The Wikidata Query service is now officially live and in production! <a href="https://t.co/D5bypZ8ZmS">https://t.co/D5bypZ8ZmS</a></p>&mdash; Wikidata (@wikidata) <a href="https://twitter.com/wikidata/status/641150926036836352">September 8, 2015</a></blockquote>

## Web Result

https://query.wikidata.org/#PREFIX%20wikibase%3A%20%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23%3E%0APREFIX%20wd%3A%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%20%0APREFIX%20wdt%3A%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0APREFIX%20rdfs%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0A%0ASELECT%20%3Fh%20%3Fcause%20%3Fhl%20%3Fcausel%20%3Fdob%20%3Fdod%20WHERE%20%7B%0A%20%20%20%3Fh%20wdt%3AP39%20wd%3AQ11696%20.%0A%20%20%20%3Fh%20wdt%3AP509%20%3Fcause%20.%0A%20%20%20%3Fh%20wdt%3AP569%20%3Fdob%20.%0A%20%20%20%3Fh%20wdt%3AP570%20%3Fdod%20.%0A%20%20%0A%20%20OPTIONAL%20%20%7B%20%20%0A%20%20%20%20%20%3Fh%20rdfs%3Alabel%20%3Fhl%20filter%20(lang(%3Fhl)%20%3D%20%22en%22)%20.%0A%20%20%7D%20%0A%20%20OPTIONAL%20%7B%20%0A%20%20%20%20%20%3Fcause%20rdfs%3Alabel%20%3Fcausel%20filter%20(lang(%3Fcausel)%20%3D%20%22en%22).%20%0A%20%20%7D%0A%7D%0A

## Issues

* multiple data of birth entries, e.g. Thomas Jefferson: https://www.wikidata.org/wiki/Q11812
* missing data for cause of death, e.g. Warren G. Harding: https://www.wikidata.org/wiki/Q35286

In [13]:
import requests

query = '''PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?h ?cause ?hl ?causel ?dob ?dod WHERE {
   ?h wdt:P39 wd:Q11696 .
   ?h wdt:P509 ?cause .
   ?h wdt:P569 ?dob .
   ?h wdt:P570 ?dod .
  
  OPTIONAL  {  
     ?h rdfs:label ?hl filter (lang(?hl) = "en") .
  } 
  OPTIONAL { 
     ?cause rdfs:label ?causel filter (lang(?causel) = "en"). 
  }
}'''

url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
params = {
    'query': query,
    'format': 'json'
}

data = requests.get(url, params=params).json()

In [14]:
for item in data['results']['bindings']:
    print(item)
    print(item['hl']['value'])
    print(item['causel']['value'])

{'causel': {'type': 'literal', 'xml:lang': 'en', 'value': 'cholera'}, 'cause': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q12090'}, 'h': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q11891'}, 'hl': {'type': 'literal', 'xml:lang': 'en', 'value': 'James K. Polk'}, 'dob': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1795-11-02T00:00:00Z'}, 'dod': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1849-06-15T00:00:00Z'}}
James K. Polk
cholera
{'causel': {'type': 'literal', 'xml:lang': 'en', 'value': 'heart attack'}, 'cause': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q12152'}, 'h': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q36023'}, 'hl': {'type': 'literal', 'xml:lang': 'en', 'value': 'Calvin Coolidge'}, 'dob': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1872-07-04T00:00:00Z'}, 'dod': {'datatype': 'http://www.w3.org/

KeyError: 'causel'